# Luxury tourism

```{admonition} Disclaimer
:class: warning

This case study is a preliminary exploration intended only to illustrate a possible use case of DataBlox-OD. The insights presented here are derived from a limited subset of our GPS data and should not be considered conclusive of actual trends.

The views expressed here are those of the authors and do not necessarily reflect the views and policies of the Asian Development Bank (ADB) or its Board of Governors or the governments they represent.
```

Load the necessary libraries.

In [1]:
import os

import geopandas as gpd
import numpy as np
import pandas as pd

import _luxury_tourism_util as util

pd.set_option("diWsplay.max_rows", 5)

%load_ext autoreload
%autoreload 2

Folder names

In [2]:
SAMPLE_DATA_DIRECTORY = os.path.join("..", "sample_data")
SAMPLE_OUTPUT_DIRECTORY = os.path.join("..", "sample_output")
CASE_STUDY_DATA_DIRECTORY = os.path.join("..", "case_study_data")
CASE_STUDY_OUTPUT_DIRECTORY = os.path.join("..", "case_study_output")

BOUNDARIES_DIRECTORY = os.path.join(SAMPLE_DATA_DIRECTORY, "boundaries")
TOURISM_DIRECTORY = os.path.join(SAMPLE_DATA_DIRECTORY, "tourism")
STAYPOINTS_DIRECTORY = os.path.join(SAMPLE_OUTPUT_DIRECTORY, "tourism", "staypoints")
ANNOTATED_TOURISM_TRIPS_DIRECTORY = os.path.join(
    SAMPLE_OUTPUT_DIRECTORY, "tourism", "annotated_tourism_trips"
)
RELATIVE_WEALTH_ANNOTATED_TOURISM_TRIPS_DIRECTORY = os.path.join(
    CASE_STUDY_OUTPUT_DIRECTORY, "relative_wealth_annotated_tourism_trips"
)
LUXURY_TOURISM_TRIPS_DIRECTORY = os.path.join(
    CASE_STUDY_OUTPUT_DIRECTORY, "luxury_tourism_trips"
)

os.makedirs(RELATIVE_WEALTH_ANNOTATED_TOURISM_TRIPS_DIRECTORY, exist_ok=True)
os.makedirs(LUXURY_TOURISM_TRIPS_DIRECTORY, exist_ok=True)

Administrative boundaries of Thailand

In [3]:
adm1 = gpd.read_file(os.path.join(BOUNDARIES_DIRECTORY, "thailand_adm1.geojson"))
adm3 = gpd.read_file(os.path.join(BOUNDARIES_DIRECTORY, "thailand_adm3.geojson"))

<hr>

## I. Data preparation

We start by annotating the visitors' staypoints with their relative wealth based on Meta's [relative wealth index map](https://www.pnas.org/doi/10.1073/pnas.2113658119), which has a spatial resolution of 1 km.

In [4]:
util.annotate_staypoints_with_relative_wealth(
    ANNOTATED_TOURISM_TRIPS_DIRECTORY,
    RELATIVE_WEALTH_ANNOTATED_TOURISM_TRIPS_DIRECTORY,
    os.path.join(CASE_STUDY_DATA_DIRECTORY, "tha_relative_wealth_index.csv"),
)

Filter the visitors to include only luxury visitors &mdash; those who visited a place with relative wealth in the 99.9<sup>th</sup> percentile.

In [5]:
util.identify_trips_with_visit_to_wealthy_area(
    RELATIVE_WEALTH_ANNOTATED_TOURISM_TRIPS_DIRECTORY, LUXURY_TOURISM_TRIPS_DIRECTORY
)

From the previous tutorials, the visitors' staypoints have already been annotated with nearby main tourist attractions, transportation hubs, accommodation, and food and beverage establishments. Here we add additional annotations to broaden the range of points of interest and include boat storages, exhibition centers, malls, and nightlife activity spots, among others.

In [6]:
util.annotate_tourism_trips_with_addl_points_of_interest(
    LUXURY_TOURISM_TRIPS_DIRECTORY,
    points_of_interest=util.process_openstreetmap(
        os.path.join(
            SAMPLE_DATA_DIRECTORY,
            "tourism",
            "openstreetmap",
            "hotosm_tha_points_of_interest_polygons_geojson.geojson",
        )
    ),
)

<hr>

## II. Illustrating trips of luxury visitors

```{admonition} Disclaimer
:class: warning

This case study includes names of commercial establishments only for purposes of discussion and illustration. Their inclusion does not in any way imply endorsement by ADB or its affiliates.
```

To illustrate trips of luxury visitors, we consider two trips made by device ID `1BC36F71-7A9F-42A3-A2AC-F1282680C4E6`.

In [7]:
luxury_tourism = gpd.read_parquet(
    os.path.join(
        LUXURY_TOURISM_TRIPS_DIRECTORY, "1BC36F71-7A9F-42A3-A2AC-F1282680C4E6.parquet"
    )
)

### October 2024 trip: Two-night stay in Phuket with excursions to Phangnga

In [8]:
luxury_tourism.loc[51]

,time_arrive,time_depart,num_pings_in_staypoint,num_minutes_stayed,geometry,device_id,h3_cell,ADM1_EN,ADM2_EN,ADM3_EN,...,nearby_food_and_beverages_names_en,nearby_food_and_beverages_names_th,nearby_food_and_beverages_types,rwi,score_percentile,nearby_addl_points_of_interest_osm_ids,nearby_addl_points_of_interest_names,nearby_addl_points_of_interest_names_en,nearby_addl_points_of_interest_names_th,nearby_addl_points_of_interest_types
trip_id,,,,,,,,,,,,,,,,,,,,,
51,2024-10-08 05:38:45,2024-10-08 06:41:22,3,62.616667,POINT (99.02519 18.78633),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,886482c613fffff,Chiang Mai,Chiang Mai#Mueang Chiang Mai,Chiang Mai#Mueang Chiang Mai#Nong Pa Khrang,...,"[Café Amazon, None, None, Shewe Cafe, Kuai Tia...","[คาเฟ่ อเมซอน, None, None, None, ก๋วยเตี่ยวเนื...","[cafe, cafe, cafe, cafe, restaurant, restauran...",1.333,99.532696,[122605036.0],[ศูนยทรัพยากรการเรียนรูสิรินธร],[Sirindhorn Learning Resources Center],[None],[library]
51,2024-10-08 07:38:22,2024-10-08 10:20:46,9,162.400000,POINT (98.30577 8.10085),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,886422c251fffff,Phuket,Phuket#Thalang,Phuket#Thalang#Sakhu,...,"[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[fast_food, restaurant, restaurant, restaurant...",0.720,92.019832,[nan],[None],[None],[None],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2024-10-10 13:33:19,2024-10-10 14:55:57,17,82.633333,POINT (100.75101 13.68968),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,8864a495adfffff,Samut Prakan,Samut Prakan#Bang Phli,Samut Prakan#Bang Phli#Nong Prue,...,[None],[None],[None],1.071,98.134535,[nan],[None],[None],[None],[None]
51,2024-10-10 17:22:20,2024-10-10 22:37:50,16,315.500000,POINT (99.02506 18.78658),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,886482c613fffff,Chiang Mai,Chiang Mai#Mueang Chiang Mai,Chiang Mai#Mueang Chiang Mai#Nong Pa Khrang,...,"[Café Amazon, None, None, Shewe Cafe, Kuai Tia...","[คาเฟ่ อเมซอน, None, None, None, ก๋วยเตี่ยวเนื...","[cafe, cafe, cafe, cafe, restaurant, restauran...",1.333,99.532696,[122605036.0],[ศูนยทรัพยากรการเรียนรูสิรินธร],[Sirindhorn Learning Resources Center],[None],[library]


**Day 1**
- Morning:
    - The time between their last ping from their district of residence (Mueang Chiang Mai, Chiang Mai) to their first ping from Phuket is roughly an hour, suggesting that they arrived by plane. This is also supported by the fact that their first ping is close to Phuket International Airport.
    - Up until noon, they stayed in Thalang, Phuket, close to some boutique and luxury resort hotels (e.g., The Slate, which is among [2024 Tatler Best Asia 100 Hotels](https://www.tatlerasia.com/places/the-slate-phuket)).

- Afternoon:
    - They made a three-hour excursion to Takua Pa, Phangnga.

- Night:
    - They spent roughly an hour in Mueang Phuket, Phuket, close to cultural tourist attractions such as Bang Niew Shrine and Phuket Thai Hua Museum.
    - Most of their evening was spent in Kathu, Phuket, close to Kalim Beach, a couple of nightclubs, and some fine dining restaurants (e.g., L'Arôme by the Sea, which is listed in the [MICHELIN Guide](https://guide.michelin.com/en/phuket-region/phuket/restaurant/l-arome-by-the-sea)).

**Day 2**
- Morning to afternoon:
    - They went on an excursion to Thai Mueang, Phangnga, from 8 a.m. to 6 p.m. Their staypoints are close to the Love Andamar Pier and the Chulabhorn Marine Park Conservatory Centre.
- Night:
    - Similar to the previous night, they spent over an hour in Mueang Phuket, Phuket, but, this time, close to Chalong Bay (Ao Chalong), diving centers, and yacht clubs.
    - Again, most of their evening was spent in Kathu, Phuket, close to Kalim Beach, a couple of nightclubs, and some fine dining restaurants (e.g., L'Arôme by the Sea, which is listed in the [MICHELIN Guide](https://guide.michelin.com/en/phuket-region/phuket/restaurant/l-arome-by-the-sea)).

**Day 3**
- Morning:
    - They spent over an hour and a half in Mueang Phuket, Phuket, close to cultural tourist attractions such as Bang Niew Shrine and Phuket Thai Hua Museum.
    - Their last ping from Phuket was recorded at around noon, close to Phuket International Airport.
- Afternoon:
    - They recorded a ping from Bang Phli, Samut Prakan, close to Suvarnabhumi Airport. It is possible that they took a connecting flight back to ther residence, and this is a layover.
    - They arrived back in their district of residence (Mueang Chiang Mai, Chiang Mai).

Throughout their two-night trip, the following are the main tourist attractions in Phuket and Phangnga that they may have visited (main tourist attractions refer to those listed on the website of the Tourism Authority of Thailand):

In [9]:
places = util.identify_places_visited(
    "1BC36F71-7A9F-42A3-A2AC-F1282680C4E6",
    ["Phuket", "Phangnga"],
    LUXURY_TOURISM_TRIPS_DIRECTORY,
    os.path.join(TOURISM_DIRECTORY, "thailand_tourist_attractions.parquet"),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_sea_ports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_airports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY, "openstreetmap", "hotosm_tha_railways_lines_geojson.geojson"
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_points_of_interest_polygons_geojson.geojson",
    ),
    adm1,
    adm3,
)
util.plot_visited_places(
    places[
        places["category"].isin(
            ["Cultural", "Services", "Entertainment", "Nature", "seaport"]
        )
    ],
    zoom_start=11,
)

### November 2024 trip: Overnight nature trip to Rayong followed by overnight urban trip to Bangkok

In [10]:
luxury_tourism.loc[63]

,time_arrive,time_depart,num_pings_in_staypoint,num_minutes_stayed,geometry,device_id,h3_cell,ADM1_EN,ADM2_EN,ADM3_EN,...,nearby_food_and_beverages_names_en,nearby_food_and_beverages_names_th,nearby_food_and_beverages_types,rwi,score_percentile,nearby_addl_points_of_interest_osm_ids,nearby_addl_points_of_interest_names,nearby_addl_points_of_interest_names_en,nearby_addl_points_of_interest_names_th,nearby_addl_points_of_interest_types
trip_id,,,,,,,,,,,,,,,,,,,,,
63,2024-11-11 19:09:56,2024-11-12 06:56:11,13,706.250000,POINT (99.0251 18.78636),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,886482c613fffff,Chiang Mai,Chiang Mai#Mueang Chiang Mai,Chiang Mai#Mueang Chiang Mai#Nong Pa Khrang,...,"[Café Amazon, None, None, Shewe Cafe, Kuai Tia...","[คาเฟ่ อเมซอน, None, None, None, ก๋วยเตี่ยวเนื...","[cafe, cafe, cafe, cafe, restaurant, restauran...",1.333,99.532696,[122605036.0],[ศูนยทรัพยากรการเรียนรูสิรินธร],[Sirindhorn Learning Resources Center],[None],[library]
63,2024-11-13 07:08:08,2024-11-13 08:39:22,9,91.233333,POINT (101.27472 12.72576),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,8865884d1bfffff,Rayong,Rayong#Mueang Rayong,Rayong#Mueang Rayong#Nam Khok,...,[None],[None],[None],0.913,96.252569,[nan],[None],[None],[None],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2024-11-15 11:59:54,2024-11-15 13:03:19,4,63.416667,POINT (100.75543 13.69044),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,8864a495adfffff,Samut Prakan,Samut Prakan#Bang Phli,Samut Prakan#Bang Phli#Nong Prue,...,[None],[None],[None],1.071,98.134535,[nan],[None],[None],[None],[None]
63,2024-11-15 15:32:57,2024-11-15 17:25:06,11,112.150000,POINT (99.02508 18.78607),1BC36F71-7A9F-42A3-A2AC-F1282680C4E6,886482c613fffff,Chiang Mai,Chiang Mai#Mueang Chiang Mai,Chiang Mai#Mueang Chiang Mai#Nong Pa Khrang,...,"[Café Amazon, None, None, Shewe Cafe, Kuai Tia...","[คาเฟ่ อเมซอน, None, None, None, ก๋วยเตี่ยวเนื...","[cafe, cafe, cafe, cafe, restaurant, restauran...",1.333,99.532696,[122605036.0],[ศูนยทรัพยากรการเรียนรูสิรินธร],[Sirindhorn Learning Resources Center],[None],[library]


**Day 1**
- Morning:
    - They arrived in Mueang Rayong, Rayong, at around 7:00 a.m.
- Afternoon to evening:
    - From afternoon to evening, they remained in Mueang Rayong, Rayong, but moved to staypoints in the subdistrict of Phe that are closer to nature tourist attractions such as Ao Wong Dean, Ao Tubtim, and Ko Samet. They spent roughly 11 hours in this area, which is located on the gulf coast of Eastern Thailand.

**Day 2**
- Morning:
    - They moved to the subdistrict of Choeng Noen, still in Mueang Rayong, Rayong. 
- Afternoon:
    - They arrived in Ratchathewi, Bangkok, and briefly stayed there for around 40 minutes, most likely to check in to their accommodation. Nearby food and beverage establishments include a notably wide range of restaurants serving international cuisine, along with biergartens and nightclubs. OpenStreetMap also records over a hundred hotels within the 2-km radius, including luxury hotels (e.g., Sheraton Grande Sukhumvit).
- Night:
    - Most of their evening was evenly spent between Pathum Wan and Vadhana, Bangkok, close to cultural attractions (e.g., Erawan Shrine), shopping centers (e.g., Siam Paragon, one of the largest malls in Thailand), galleries (e.g., Madame Tussauds Bangkok), and parks (e.g., Lumpini Park).
    - For the remainder of the night, they stayed in Ratchathewi, Bangkok, at roughly the same staypoint where they arrived that afternoon. This supports the earlier inference of their accommodation likely being located in this area.

**Day 3** 
- Morning:
    - They stayed in the same staypoint where they were the previous night.
- Afternoon:
    - At noon, they recorded a ping from Bang Phli, Samut Prakan, close to Suvarnabhumi Airport. It is possible that they took a connecting flight back to ther residence, and this is a layover.
    - They arrived back in their district of residence (Mueang Chiang Mai, Chiang Mai).

Throughout their overnight nature trip to Rayong, the following are the main tourist attractions that they may have visited:

In [11]:
places = util.identify_places_visited(
    "1BC36F71-7A9F-42A3-A2AC-F1282680C4E6",
    ["Rayong"],
    LUXURY_TOURISM_TRIPS_DIRECTORY,
    os.path.join(TOURISM_DIRECTORY, "thailand_tourist_attractions.parquet"),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_sea_ports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_airports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY, "openstreetmap", "hotosm_tha_railways_lines_geojson.geojson"
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_points_of_interest_polygons_geojson.geojson",
    ),
    adm1,
    adm3,
)
util.plot_visited_places(
    places[
        places["category"].isin(
            ["Cultural", "Services", "Entertainment", "Nature", "seaport"]
        )
    ],
    zoom_start=14,
)

Throughout their overnight urban trip to Bangkok, the following are the main tourist attractions that they may have visited:

In [12]:
places = util.identify_places_visited(
    "1BC36F71-7A9F-42A3-A2AC-F1282680C4E6",
    ["Bangkok"],
    LUXURY_TOURISM_TRIPS_DIRECTORY,
    os.path.join(TOURISM_DIRECTORY, "thailand_tourist_attractions.parquet"),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_sea_ports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_airports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY, "openstreetmap", "hotosm_tha_railways_lines_geojson.geojson"
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_points_of_interest_polygons_geojson.geojson",
    ),
    adm1,
    adm3,
)
util.plot_visited_places(
    places[
        places["category"].isin(
            ["Cultural", "Services", "Entertainment", "Nature", "seaport"]
        )
    ],
    zoom_start=14,
)

<hr>

## II. Profiling luxury visitors and their visits

Identify where most luxury visitors come from.

In [13]:
residences = util.profile_provinces_of_residence_of_visitors(
    LUXURY_TOURISM_TRIPS_DIRECTORY
)
for item in residences.most_common(5):
    print(
        f"{item[0]}:",
        item[1],
        f"({item[1] / sum(residences.values()) * 100}%)",
    )

  0%|          | 0/1202 [00:00<?, ?it/s]

Bangkok: 345 (28.70216306156406%)
Pathum Thani: 95 (7.903494176372712%)
Nonthaburi: 61 (5.074875207986689%)
Samut Prakan: 60 (4.9916805324459235%)
Chon Buri: 51 (4.242928452579035%)


Investigate whether luxury trips are multi-destination trips or more focused at only one area.

In [14]:
util.profile_num_destination_of_trips(LUXURY_TOURISM_TRIPS_DIRECTORY)

  0%|          | 0/1202 [00:00<?, ?it/s]

Province
Mean: 1.9750415973377704
Median: 2.0
Min: 1
Max: 11
District
Mean: 3.509983361064892
Median: 3.0
Min: 1
Max: 23


Identify which provinces are the most frequently visited in luxury trips, as well as the lengths of the visits.

In [15]:
profile_destination = util.profile_visited_provinces(LUXURY_TOURISM_TRIPS_DIRECTORY)

  0%|          | 0/1202 [00:00<?, ?it/s]

In [16]:
profile_destination.sort_values(("num_nights_stayed", "mean"), ascending=False).head(5)

num_nights_stayed                         
                    count      mean min max median
ADM1_EN                                           
Trat                    1  2.000000   2   2    2.0
Pattani                 4  1.750000   0   4    1.5
Phayao                  7  1.571429   0  10    0.0
Roi Et                 11  1.363636   0   6    1.0
Phrae                  14  1.357143   0  10    0.0

In [17]:
profile_destination.sort_values(("num_nights_stayed", "count"), ascending=False).head(5)

num_nights_stayed                         
                         count      mean min max median
ADM1_EN                                                
Bangkok                    716  0.687151   0  21    0.0
Chon Buri                  208  0.485577   0   8    0.0
Nonthaburi                 193  0.559585   0  32    0.0
Pathum Thani               148  0.250000   0   6    0.0
Samut Prakan               126  0.388889   0   9    0.0

`````{admonition} Insights
:class: tip
* Most luxury visitors come from Bangkok (29%), followed by Pathum Thani (8%).
* On average, they visit 2 provinces or 4 districts per luxury trip.
* The most visited luxury trip destination is Bangkok, followed by Chon Buri. However, visits there tend to be same-day trips (excursions). 
* The luxury trip destinations where visitors tend to stay the longest on average are Trat (2 nights) and Pattani (1.75 nights), both of which are coastal provinces bordering the Gulf of Thailand.
`````